참고 url : https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

In [ ]:
!pip install transformers==2.9.0 
!pip install pytorch_lightning==0.7.5
!pip install sentencepiece==0.1.91


런타임 다시 시작 눌러야할 수 있음  
만일 tokenizer Nonetype 에러시 런타임 다시 시작

In [ ]:
# model.generate(pieces)
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader



config = T5Config.from_json_file('/content/drive/MyDrive/Colab/etri_et5/config.json')
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Colab/etri_et5/pytorch_model.bin", config=config)

token_config = T5Config.from_json_file('/content/drive/MyDrive/Colab/etri_et5/tokenizer_config.json')vocab_file = '/content/drive/MyDrive/Colab/etri_et5/spiece.model'tokenizer = T5Tokenizer(vocab_file=vocab_file, config=token_config)

In [ ]:
tokenizer

In [ ]:
pieces = tokenizer.tokenize('"대졸 신입사원 연봉은 얼마인가요"란 질문에 ㈜한화 인재개발팀의 전정수 대리는 "㈜한화 화약/방산 부문 대졸 학사기준 초임은 약 4500~4600만원 수준입니다. 경영성과금 및 수당(자격,기술,지역근무 등)은 제외된 기본 연봉입니다"이라고 솔직하게 연봉을 공개했다.채용에 대한 실시간 질의응답이 열린 곳은 대학가 캠퍼스 리크루팅이 아닌 인터넷.㈜한화는 3일 오후 4시부터 90분간 카카오TV 라이브 플랫폼을 이용해 온라인 라이브 채용설명회를 열었다.이날 채용설명회에선 평소 밝히기를 꺼려왔던 취준생들의 궁금증에 대해 속 시원히 답변해 준 시간이었다."연구개발부서의 학사출신 비중이 어느정도 인가요?"란 질문에는 "20~30%가 학사출신 연구원"이라며 "학사라도 면접때 전공 역량과 연구개발에 대한 열정을 보여준다면 충분히 입사가 가능할 것"이라고 답했다.자기소개서 작성과 관련해서 전 대리는 "자소서에 나와있는 글자수는 최대 글자수이기 때문에 반드시 그 글자수에 맞추지는 않아도 된다"고 설명했다.또한, "㈜한화는 글로벌 회사 도약을 위해 영어 커뮤니케이션 능력이 뛰어나면 우대하고 있다"고 말했다.채용설명회는 한화그룹의 △방위산업 △주요사업 △인사·채용 △주요직무 소개 △질의응답 등의 순서로 진행됐다.㈜한화는 지난해 하반기 공채부터 온라인 라이브 설명회를 열고 있다.㈜한화 관계자는 "실제 캠퍼스 리크루팅보다 20배이상의 구직자들이 몰려 뜨거운 반응을 나타냈다"며 "앞으로도 지속적으로 온라인 채용설명회를 개최할 방침"이라고 말했다.지난달부터 상반기 공채를 시작한 ㈜한화 화학·방산부문은 오는 7일까지 신입사원을 모집한다.지난해 11월 화학·방산부문의 통합후 첫 공채다.용분야는 △경영관리 △생산품질관리 △연구개발 등이다.㈜한화 무역부문도 수출입무역·재경직 등에서 신입사원을 채용한다.영어능통자, 제2외국어 가능자, 이공계출신의 수출입영업 지원자 등은 우대한다.회사생활, 직무 등에 대한 자세한 내용은 ㈜한화 블로그를 통해서도 확인 할 수 있다.▷연봉이 얼마인가요?" (주)한화 화약/방산 부문 대졸 학사기준 초임은 약 4,500~4,600만원 수준입니다. 단, 경영성과금 및 수당(자격,기술,지역근무 등)은 제외된 기본 연봉입니다."▷학사는 연구개발 직무에 지원할 수 없나요?"충분히 지원할 수 있습니다. 현재 재직중인 연구개발 분들 중 20~30%가 학사분이시며, 실무 면접에서 전공관련 역량 및 연구개발에 대한 강한 열의를 보여주신다면 충분히 합격 가능합니다."▷공고에 기재된 적합 전공자가 아닌 경우 지원이 불가능한가요?"공고에 기재된 적합 전공은 지원자의 선택을 돕기 위해 직무와 관련성이 높은 전공을 기재한 것이며, 본인의 전공이 포함되지 않더라도 지원 가능합니다."▷작년에 최종 면접에서 탈락한 사람입니다.불이익이 있나요?"재지원에 따른 불이익은 전혀 없습니다. 실제로 최종면접에서 탈락하였음에도 불구하고, 재지원하여 최종합격한 사례도 꽤 있고 더 회사생활을 만족하며 지내고 계십니다. (주)한화 입사에 대한 간절함과 열정이 더 중요한 부분입니다."▷자소서 글자수를 맞춰야 하나요?"자소서에 나와있는 글자수는 최대 글자수이며, 반드시 그 글자수에 맞추지는 않아도 됩니다."▷우대하는 자격증은요?"(주)한화는 글로벌 회사로 도약하기 위해 노력하고 있기에, 영어관련 성적이 있으시면 좋습니다. 각 직무별로 우대하는 자격사항들이 달라 채용 세부공고사항에서 한번 더 꼼꼼히 확인해주세요.(ex. 재경직무: 회계관련 자격증, 생산관리직무: 화약류 제조/관리기사 등)" (끝) / trues@hankyung.com모바일한경은 프리미엄 채널입니다.')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
class CustomDataset:

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:


def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in tqdm(enumerate(loader, 0)):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            pass
            
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
model.to(device)

In [ ]:
config.MAX_LEN = 512
config.SUMMARY_LEN = 150 
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 3        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)

In [ ]:
train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

In [ ]:
!nvidia-smi


In [ ]:
import pandas as pd
train_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/extract_data/train_data.csv')[['document','label']]
validation_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/extract_data/validation_data.csv')[['document','label']]

In [ ]:
train_dataset.columns = ['text','ctext']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, **train_params)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
for epoch in range(config.TRAIN_EPOCHS):
    print (1)
    train(epoch, tokenizer, model, device, training_loader, optimizer)

In [ ]:


validation_dataset.columns = ['text','ctext']
validation_dataset.ctext = 'summarize: ' + validation_dataset.ctext

val_set = CustomDataset(validation_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

val_loader = DataLoader(val_set, **val_params)

for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

final_df